In [21]:
# Установка зависимостей (один раз)
%pip install -q openai python-docx

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pandas

     ---------------------------------------- 11.3/11.3 MB 8.0 MB/s eta 0:00:00
     ------------------------------------- 347.8/347.8 kB 10.9 MB/s eta 0:00:00
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
     ---------------------------------------- 12.9/12.9 MB 9.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

In [1]:
import pandas as pd

# Укажите путь к файлу (на Windows удобно использовать raw-строку r"...")
csv_path = r"C:\Users\USER\progects\Africa_analyse\problems_study\tanzania_problems_3000.csv"  # например: r"C:\Users\USER\Downloads\data.csv"

df = pd.read_csv(csv_path, sep=';', encoding='utf-8-sig', on_bad_lines='skip')  # при необходимости: sep=';', encoding='utf-8-sig' или 'cp1251'
print(f"Размер датафрейма: {df.shape[0]} строк, {df.shape[1]} столбцов")
df.head()

Размер датафрейма: 2235 строк, 46 столбцов


,№,Дата,Тип,Заголовок,Сообщение,Автораспознанный текст,Ссылка,Площадка,Тип площадки,Автор,...,Категории,Токсичность,Статус на площадке,Обработано,Избранное,Примечание,Мнения,Объекты,Теги,Id сообщения
0,1,28.09.2025 12:33,Пост,"Sept. 27, 2025: Vayelech (and He went) 2nd ser...",**DOWNLOAD THE NOTES HERE**\nhttps://esm.us/wp...,Расшифровки\r\nТранскрипт:Israel adm [Music] [...,http://www.youtube.com/watch?v=xrccfC6y3FY,YouTube,Блог,El Shaddai Ministries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,150111353791
1,2,28.09.2025 11:15,Пост,AIC KIPKORGOT SUNDAY SERVICE,CHURCH SERVICE,Расшифровки\r\nТранскрипт:Heat. Heat. Heat. He...,http://www.youtube.com/watch?v=Iozeui_MT7o,YouTube,Блог,PineHut,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,150089964026
2,3,28.09.2025 10:55,Пост,NaN,Abdul Rahman knew at this point that he had tw...,NaN,https://telegram.me/sayyidazeem/5694,Telegram,Мессенджер,Sayyid's newsletter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,150107674911
3,4,27.09.2025 22:27,Пост,Business Opportunity with Rap Motors,#franchiseindia #investmentopportunity #starty...,Тексты с изображений\r\nindia*s leading 3 whee...,http://www.youtube.com/watch?v=264uwhBWoU8,YouTube,Блог,Franchise India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,150057126221
4,5,27.09.2025 20:30,Пост,BREAKING: Traoré & Erdoğan sign $5.2B Ankara d...,"95,62,72,82... BREAKING: Traoré & Erdoğan sign...",Тексты с изображений\r\nankara signs big s5.2b...,http://www.youtube.com/watch?v=Lz_0JOBzHUw,YouTube,Блог,Africa Politics,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150055391153


In [ ]:
# удаляем дубликаты по столбцу "Заголовок", если он есть
df.drop_duplicates(subset=['Заголовок'], inplace=True)
df.reset_index(drop=True, inplace=True) 
print(f"Размер датафрейма после удаления дубликатов: {df.shape[0]} строк, {df.shape[1]} столбцов")


Размер датафрейма после удаления дубликатов: 1930 строк, 46 столбцов


In [ ]:
%pip install -q openai python-dotenv tqdm


In [1]:
import os
from dotenv import load_dotenv
from tqdm import tqdm
import pandas as pd

# загружаем переменные окружения, где хранится OPENAI_API_KEY
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "OPENAI_API_KEY не найден в .env"



In [22]:
# список проблем
FRENCH_PROBLEMS = [
"Unemployment", "Informal sector", "Precarious employment", "Inflation", "Poverty", 
"Inequality", "Undernourishment", "Malnutrition", "Access to water", "Sanitation",
"Inadequate housing", "Urbanization", "Pollution", "Deforestation", "Climate change",
"Drought", "Floods", "Corruption", "Poor governance", "Political instability", "Insecurity",
"Conflicts", "Forced displacement", "Child labor", "Early marriage", "Domestic violence", 
"Discrimination", "Access to education", "Illiteracy", "Youth underemployment", 
"Weak healthcare system", "Infant mortality", "Epidemics", "Cost of healthcare", 
"Medicine shortage", "Degraded infrastructure", "Limited transportation", "Lack of electricity", 
"Energy shortage", "Limited internet access", "Brain drain", "Low investment", "Economic dependence", 
"Public debt", "Limited exports", "Price fluctuation", "Unstable market", "Weak tourism", 
"Mining exploitation", "Unproductive agriculture"
]



In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

SYSTEM_PROMPT = (
"""
"Ты — классификатор. Дай ровно один ярлык из предоставленного набора (на французском языке), \
    который является самым релевантным и наиболее часто встречающимся в данном тексте.\
Если ни один вариант не подходит, ответь точно: "нет". Отвечай только ярлыком или "нет"."
"""
)

USER_TEMPLATE = (
"""
Список допустимых ярлыков: {labels}

Текст:
{text}

Верни ТОЛЬКО тот ярлык, который наиболее часто встречается / 
наиболее уместен среди списка, ИЛИ "нет", если ни один не подходит.
"""
)

def classify_text_to_label(text: str, labels: list[str]) -> str:
    if not isinstance(text, str) or not text.strip():
        return "нет"
    user_prompt = USER_TEMPLATE.format(labels=labels, text=text[:2000])
    completion = client.chat.completions.create(
        model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0,
        max_tokens=10,
    )
    answer = completion.choices[0].message.content.strip().strip(" .,:;\"'`")
    if answer.lower() == "нет":
        return "нет"
    normalized = {lbl.lower(): lbl for lbl in labels}
    key = answer.lower()
    return normalized.get(key, "нет")



In [24]:
print(df.shape)
csv_path

(2235, 46)


'C:\\Users\\USER\\progects\\Africa_analyse\\problems_study\\tanzania_problems_3000.csv'

In [27]:
# читаем исходный CSV с колонкой "сообщение"
# замените путь при необходимости
# csv_path = r"C:\Users\USER\progects\Africa_analyse\problems_study\Togo_nets.csv"
# df_full = pd.read_csv(csv_path, sep=';')

# если нужно, то можно сделать выборку
# df = df_full.sample(frac=0.01)
# df = df_full


# применяем классификацию
labels = FRENCH_PROBLEMS
preds = []
for text in tqdm(df["Сообщение"].fillna("").astype(str), desc="Classifying"):
    preds.append(classify_text_to_label(text, labels))

df["проблема"] = preds

# сохраняем результат
out_path = csv_path.replace('.csv', '_classified.csv')
df.to_csv(out_path, index=False, sep=';')
out_path


Classifying: 100%|██████████| 1930/1930 [14:09<00:00,  2.27it/s]


'C:\\Users\\USER\\progects\\Africa_analyse\\problems_study\\tanzania_problems_3000_classified.csv'

In [5]:
res_table = df['проблема'].value_counts()
res_table = res_table[(res_table > 1) & (res_table.index != 'нет')]
res_table



проблема
Poverty                  35
Climate change           22
Corruption               15
Inflation                 9
Unemployment              6
Access to water           6
Sanitation                5
Malnutrition              5
Insecurity                4
Inequality                4
Pollution                 4
Floods                    4
Inadequate housing        4
Mining exploitation       3
Child labor               2
Precarious employment     2
Access to education       2
Name: count, dtype: int64

In [29]:
os.listdir(r"C:\Users\USER\progects\Africa_analyse\problems_study")

['.env',
 '.git',
 '.gitignore',
 'main.ipynb',
 'README.md',
 'tanzania_problems.csv',
 'tanzania_problems_3000.csv',
 'tanzania_problems_3000_classified.csv',
 'tanzania_problems_classified.csv',
 'Togo_nets.csv',
 'Togo_nets_classified.csv',
 'Togo_nets_classified_filt.csv',
 'Togo_nets_problem_table.csv',
 'togo_problems_report_20250814_1205.docx',
 'togo_problems_report_20250814_1214.docx',
 'togo_problems_report_20250814_1222.docx',
 'venv']

In [13]:
df = pd.read_csv(r"C:\Users\USER\progects\Africa_analyse\problems_study\tanzania_problems_3000_classified.csv", sep=';')
df[:3]

,№,Дата,Тип,Заголовок,Сообщение,Автораспознанный текст,Ссылка,Площадка,Тип площадки,Автор,...,Токсичность,Статус на площадке,Обработано,Избранное,Примечание,Мнения,Объекты,Теги,Id сообщения,проблема
0,1,28.09.2025 12:33,Пост,"Sept. 27, 2025: Vayelech (and He went) 2nd ser...",**DOWNLOAD THE NOTES HERE**\nhttps://esm.us/wp...,Расшифровки\r\nТранскрипт:Israel adm [Music] [...,http://www.youtube.com/watch?v=xrccfC6y3FY,YouTube,Блог,El Shaddai Ministries,...,NaN,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,150111353791,нет
1,2,28.09.2025 11:15,Пост,AIC KIPKORGOT SUNDAY SERVICE,CHURCH SERVICE,Расшифровки\r\nТранскрипт:Heat. Heat. Heat. He...,http://www.youtube.com/watch?v=Iozeui_MT7o,YouTube,Блог,PineHut,...,NaN,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,150089964026,нет
2,3,28.09.2025 10:55,Пост,NaN,Abdul Rahman knew at this point that he had tw...,NaN,https://telegram.me/sayyidazeem/5694,Telegram,Мессенджер,Sayyid's newsletter,...,NaN,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,150107674911,нет


In [2]:
# перевод проблем на русский с французского с помощью словаря

# problems_dict = {
#     "Chômage": "Безработица",
#     "Secteur informel": "Неформальный сектор",
#     "Emploi précaire": "Нестабильная занятость",
#     "Inflation": "Инфляция",
#     "Pauvreté": "Бедность",
#     "Inégalités": "Неравенство",
#     "Sous-alimentation": "Недоедание",
#     "Malnutrition": "Недостаточное питание",
#     "Accès à l'eau": "Доступ к воде",
#     "Assainissement": "Санитария",
#     "Logement insalubre": "Непригодное жильё",
#     "Urbanisation": "Урбанизация",
#     "Pollution": "Загрязнение",
#     "Déforestation": "Обезлесение",
#     "Changement climatique": "Изменение климата",
#     "Sécheresse": "Засуха",
#     "Inondations": "Наводнения",
#     "Corruption": "Коррупция",
#     "Mauvaise gouvernance": "Плохое управление",
#     "Instabilité politique": "Политическая нестабильность",
#     "Insécurité": "Небезопасность",
#     "Conflits": "Конфликты",
#     "Déplacement forcé": "Вынужденное переселение",
#     "Travail des enfants": "Детский труд",
#     "Mariage précoce": "Ранние браки",
#     "Violence domestique": "Домашнее насилие",
#     "Discrimination": "Дискриминация",
#     "Accès à l'éducation": "Доступ к образованию",
#     "Analphabétisme": "Безграмотность",
#     "Sous-emploi des jeunes": "Недоиспользование труда молодёжи",
#     "Système de santé faible": "Слабая система здравоохранения",
#     "Mortalité infantile": "Детская смертность",
#     "Épidémies": "Эпидемии",
#     "Coût des soins": "Стоимость медицинских услуг",
#     "Manque de médicaments": "Недостаток лекарств",
#     "Infrastructure dégradée": "Разрушенная инфраструктура",
#     "Transport limité": "Ограниченный транспорт",
#     "Manque d'électricité": "Отсутствие электричества",
#     "Pénurie d'énergie": "Дефицит энергии",
#     "Accès limité à internet": "Ограниченный доступ к интернету",
#     "Fuite des cerveaux": "Утечка мозгов",
#     "Faible investissement": "Низкие инвестиции",
#     "Dépendance économique": "Экономическая зависимость",
#     "Dette publique": "Государственный долг",
#     "Exportations limitées": "Ограниченный экспорт",
#     "Fluctuation des prix": "Колебания цен",
#     "Marché instable": "Нестабильный рынок",
#     "Tourisme faible": "Слабый туризм",
#     "Exploitation minière": "Добыча полезных ископаемых",
#     "Agriculture peu productive": "Малопродуктивное сельское хозяйство"
# }

# переводим на русский 
problems_dict = {"Unemployment": "Безработица",
    "Informal sector": "Неформальный сектор",
    "Precarious employment": "Нестабильная занятость",
    "Inflation": "Инфляция",
    "Poverty": "Бедность",
    "Inequality": "Неравенство",
    "Undernourishment": "Недоедание",
    "Malnutrition": "Недостаточное питание",
    "Access to water": "Доступ к воде",
    "Sanitation": "Санитария",
    "Inadequate housing": "Неадекватное жильё",
    "Urbanization": "Урбанизация",
    "Pollution": "Загрязнение",
    "Deforestation": "Обезлесение",
    "Climate change": "Изменение климата",
    "Drought": "Засуха",
    "Floods": "Наводнения",
    "Corruption": "Коррупция",
    "Poor governance": "Плохое управление",
    "Political instability": "Политическая нестабильность",
    "Insecurity": "Небезопасность",
    "Conflicts": "Конфликты",
    "Forced displacement": "Вынужденное переселение",
    "Child labor": "Детский труд",
    "Early marriage": "Ранние браки",
    "Domestic violence": "Домашнее насилие",
    "Discrimination": "Дискриминация",
    "Access to education": "Доступ к образованию",
    "Illiteracy": "Безграмотность",
    "Youth underemployment": "Недоиспользование труда молодёжи",
    "Weak healthcare system": "Слабая система здравоохранения",
    "Infant mortality": "Детская смертность",
    "Epidemics": "Эпидемии",
    "Cost of healthcare": "Стоимость медицинских услуг",
    "Medicine shortage": "Недостаток лекарств",
    "Degraded infrastructure": "Разрушенная инфраструктура",
    "Limited transportation": "Ограниченный транспорт",
    "Lack of electricity": "Отсутствие электричества",
    "Energy shortage": "Дефицит энергии",
    "Limited internet access": "Ограниченный доступ к интернету",
    "Brain drain": "Утечка мозгов",
    "Low investment": "Низкие инвестиции",
    "Economic dependence": "Экономическая зависимость",
    "Public debt": "Государственный долг",
    "Limited exports": "Ограниченный экспорт",
    "Price fluctuation": "Колебания цен",
    "Unstable market": "Нестабильный рынок",
    "Weak tourism": "Слабый туризм",
    "Mining exploitation": "Добыча полезных ископаемых",
    "Unproductive agriculture": "Малопродуктивное сельское хозяйство"
}


In [4]:
problems_dict.values()

dict_values(['Безработица', 'Неформальный сектор', 'Нестабильная занятость', 'Инфляция', 'Бедность', 'Неравенство', 'Недоедание', 'Недостаточное питание', 'Доступ к воде', 'Санитария', 'Неадекватное жильё', 'Урбанизация', 'Загрязнение', 'Обезлесение', 'Изменение климата', 'Засуха', 'Наводнения', 'Коррупция', 'Плохое управление', 'Политическая нестабильность', 'Небезопасность', 'Конфликты', 'Вынужденное переселение', 'Детский труд', 'Ранние браки', 'Домашнее насилие', 'Дискриминация', 'Доступ к образованию', 'Безграмотность', 'Недоиспользование труда молодёжи', 'Слабая система здравоохранения', 'Детская смертность', 'Эпидемии', 'Стоимость медицинских услуг', 'Недостаток лекарств', 'Разрушенная инфраструктура', 'Ограниченный транспорт', 'Отсутствие электричества', 'Дефицит энергии', 'Ограниченный доступ к интернету', 'Утечка мозгов', 'Низкие инвестиции', 'Экономическая зависимость', 'Государственный долг', 'Ограниченный экспорт', 'Колебания цен', 'Нестабильный рынок', 'Слабый туризм', 'Д

In [7]:
df['problem_russ'] = df['проблема'].map(problems_dict)
df

,№,Дата,Тип,Заголовок,Сообщение,Автораспознанный текст,Ссылка,Площадка,Тип площадки,Автор,...,Статус на площадке,Обработано,Избранное,Примечание,Мнения,Объекты,Теги,Id сообщения,проблема,problem_russ
0,1,28.09.2025 12:33,Пост,"Sept. 27, 2025: Vayelech (and He went) 2nd ser...",**DOWNLOAD THE NOTES HERE**\nhttps://esm.us/wp...,Расшифровки\r\nТранскрипт:Israel adm [Music] [...,http://www.youtube.com/watch?v=xrccfC6y3FY,YouTube,Блог,El Shaddai Ministries,...,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,150111353791,нет,NaN
1,2,28.09.2025 11:15,Пост,AIC KIPKORGOT SUNDAY SERVICE,CHURCH SERVICE,Расшифровки\r\nТранскрипт:Heat. Heat. Heat. He...,http://www.youtube.com/watch?v=Iozeui_MT7o,YouTube,Блог,PineHut,...,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,150089964026,нет,NaN
2,3,28.09.2025 10:55,Пост,NaN,Abdul Rahman knew at this point that he had tw...,NaN,https://telegram.me/sayyidazeem/5694,Telegram,Мессенджер,Sayyid's newsletter,...,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,150107674911,нет,NaN
3,4,27.09.2025 22:27,Пост,Business Opportunity with Rap Motors,#franchiseindia #investmentopportunity #starty...,Тексты с изображений\r\nindia*s leading 3 whee...,http://www.youtube.com/watch?v=264uwhBWoU8,YouTube,Блог,Franchise India,...,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,150057126221,нет,NaN
4,5,27.09.2025 20:30,Пост,BREAKING: Traoré & Erdoğan sign $5.2B Ankara d...,"95,62,72,82... BREAKING: Traoré & Erdoğan sign...",Тексты с изображений\r\nankara signs big s5.2b...,http://www.youtube.com/watch?v=Lz_0JOBzHUw,YouTube,Блог,Africa Politics,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150055391153,нет,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1925,2228,03.01.2025 12:13,Пост,Kyiv is about to lower the mobilization age to...,"Happy New Year!\nDear friends, let me remind y...",Расшифровки\r\nТранскрипт:hello everyone today...,http://www.youtube.com/watch?v=OBx1h3bDPrA,YouTube,Блог,Levan Gudadze - Opinion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129559078745,нет,NaN
1926,2229,03.01.2025 11:52,Пост,2025 Outlook on Africa | Uganda joins BRICS gr...,Uganda has become the latest African country t...,Тексты с изображений\r\nsabc news 2025 outlook...,http://www.youtube.com/watch?v=vhSPZlgr3os,YouTube,Блог,SABC News,...,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,129519519066,нет,NaN
1927,2230,02.01.2025 12:06,Пост,THIS MAN! LISTEN TO HOW PRESIDENT RUTO WOWS SI...,SUBSCRIBE\n#Kenyanews​ #Ajaabunews​ #WilliamRu...,Расшифровки\r\nТранскрипт:kabut musi president...,http://www.youtube.com/watch?v=CRwXo9OfdCA,YouTube,Блог,AJAABU TV,...,NaN,NaN,NaN,NaN,NaN,Tanzania,NaN,129454822941,нет,NaN
1928,2232,01.01.2025 11:58,Пост,"S/Korea Acting Pres. Visits Seoul Memorial, Ho...",Don't forget to subscribe: https://bit.ly/2Hb8...,Расшифровки\r\nТранскрипт:[Music] it's 5:00 p....,http://www.youtube.com/watch?v=EdboXSUBOPo,YouTube,Блог,Channels Television,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129428294552,нет,NaN


In [8]:
df['problem_russ'].value_counts()

problem_russ
Бедность                            35
Изменение климата                   22
Коррупция                           15
Инфляция                             9
Безработица                          6
Доступ к воде                        6
Санитария                            5
Недостаточное питание                5
Небезопасность                       4
Неравенство                          4
Загрязнение                          4
Наводнения                           4
Неадекватное жильё                   4
Добыча полезных ископаемых           3
Детский труд                         2
Нестабильная занятость               2
Доступ к образованию                 2
Эпидемии                             1
Недоиспользование труда молодёжи     1
Обезлесение                          1
Урбанизация                          1
Дискриминация                        1
Конфликты                            1
Name: count, dtype: int64

In [9]:
# отфильтруем датафрейм, оставив только проблемы, которые встречаются более 1 раза
df_filt = df[df['проблема'].isin(res_table.index)]
df_filt.shape

(132, 48)

In [11]:
# сохраняем отфильтрованный датафрейм
df_filt.to_csv(r'C:\Users\USER\progects\Africa_analyse\problems_study\Tanzania_classified_filt.csv', index=False, sep=';')
final_problem_tables = df_filt['problem_russ'].value_counts()
final_problem_tables.to_csv(r'C:\Users\USER\progects\Africa_analyse\problems_study\Tanzania_problem_table.csv', index=False, sep=';')

In [10]:
df = df_filt

In [12]:
import os
from datetime import datetime
import pandas as pd
from openai import OpenAI
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.section import WD_SECTION
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
import re

# =============================
# Конфигурация
# =============================
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")
MAX_MESSAGES_PER_PROBLEM = 15
MAX_CHARS_PER_MESSAGE = 2000
OUTPUT_DOCX_PATH = f"Tanzania_problems_report_{datetime.now().strftime('%Y%m%d_%H%M')}.docx"

# =============================
# Проверки окружения и колонок
# =============================
if "df" not in globals():
    raise RuntimeError(
        "В памяти должен быть датафрейм 'df'. Убедитесь, что вы его подготовили в предыдущих ячейках."
    )

if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError(
        "Не обнаружен OPENAI_API_KEY. Установите переменную окружения OPENAI_API_KEY с вашим ключом."
    )

# Определяем имена колонок
problem_col = None
for col in ["проблема", "problem", "Проблема"]:
    if col in df.columns:
        problem_col = col
        break
if problem_col is None:
    raise ValueError("Не найдена колонка с типом проблемы ('проблема'/'problem').")

problem_ru_col = None
for col in ["problem_russ", "problem_ru", "проблема_ru", "проблема_rus", "проблема_рус", "проблема_перевод"]:
    if col in df.columns:
        problem_ru_col = col
        break

message_col = None
for col in ["Сообщение", "Сообщения", "сообщение", "text", "Текст"]:
    if col in df.columns:
        message_col = col
        break
if message_col is None:
    raise ValueError("Не найдена колонка с текстами сообщений ('Сообщение'/'Сообщения').")

link_col = "Ссылка" if "Ссылка" in df.columns else None

# =============================
# Вспомогательные функции
# =============================
client = OpenAI()

def add_page_numbers(doc):
    """Добавляет номера страниц в документ (включая титульную)"""
    def create_element(name):
        return OxmlElement(name)
    
    def create_attribute(element, name, value):
        element.set(qn(name), value)
        return element
    
    for section in doc.sections:
        footer = section.footer
        footer_para = footer.paragraphs[0] if footer.paragraphs else footer.add_paragraph()
        footer_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        footer_para.clear()
        run = footer_para.add_run()
        fldChar1 = create_element('w:fldChar')
        create_attribute(fldChar1, 'w:fldCharType', 'begin')
        instrText = create_element('w:instrText')
        instrText.text = "PAGE"
        fldChar2 = create_element('w:fldChar')
        create_attribute(fldChar2, 'w:fldCharType', 'end')
        run._r.append(fldChar1)
        run._r.append(instrText)
        run._r.append(fldChar2)

def add_header(doc, header_text):
    """Добавляет колонтитул с названием отчета (включая титульную)"""
    for section in doc.sections:
        header = section.header
        header_para = header.paragraphs[0] if header.paragraphs else header.add_paragraph()
        header_para.text = header_text
        header_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        for r in header_para.runs:
            r.font.size = Pt(10)
            r.font.italic = True


def normalize_message(msg: object) -> str | None:
    if pd.isna(msg):
        return None
    s = str(msg).strip()
    if len(s) > MAX_CHARS_PER_MESSAGE:
        s = s[:MAX_CHARS_PER_MESSAGE] + "..."
    return s if s else None


def build_prompt(problem_key: str, problem_ru: str | None, messages: list[str], links: list[str]):
    msgs_block = "\n\n".join([f"- {m}" for m in messages])
    problem_title = f"{problem_key}" + (f" / {problem_ru}" if problem_ru else "")
    
    links_info = ""
    if links:
        links_info = f"\n\nСсылки на источники:\n" + "\n".join([f"- {link}" for link in links])

    sys = (
        "Ты профессиональный социолог. Готовишь отчеты для государственных и международных организаций. "
        "Проанализируй высказывания жителей государства Танзания в соцсетях и опиши проблемы, которые они поднимают. "
        "Пиши на русском языке, чётко и по делу. "
        "Фокус: Танзания. Допускается упоминание других стран, если это необходимо для контекста и понимания влияния на Танзанию. Отдавай приоритет фактам и событиям; анализ своди к 1–2 предложениям на каждый пример."
    )

    usr = f"""
Тип проблемы: {problem_title}

Высказывания (до {MAX_MESSAGES_PER_PROBLEM} шт):
{msgs_block}
{links_info}

Задача:
1) Сформулируй суть ключевых проблем на основе ТОЛЬКО данных, относящихся к Танзании.
2) Подробно опиши проявления и представь 2–3 КОНКРЕТНЫЕ ПРИМЕРА с понятным контекстом: где/когда/кто/что произошло.
3) Укажи, кого это затрагивает сильнее всего (если понятно).

Ограничения:
- Упоминание других стран допустимо, если это необходимо для понимания контекста и влияния на Танзанию.
- Не используй формулировки вроде «и соседних стран».
- Не добавляй раздел «Выводы» и не пиши общий итоговый абзац по разделу.

Важно:
- Для КАЖДОГО примера приведи 1–2 цитаты (В ЦИТАТАХ МОЖНО ИСПОЛЬЗОВАТЬ ТОЛЬКО ГРАЖДАН ТАНЗАНИИ!!!, 
ИСПОЛЬЗОВАТЬ В ЦИТАТАХ СЛОВА ПРЕЗИДЕНТОВ, ЧИНОВНИКОВ ДРУГИХ СТРАН ЗАПРЕЩЕНО!!!), которые прямо иллюстрируют его суть, и сразу дай перевод на русский в скобках. Цитаты должны быть информативными (не общими) и раскрывать проблему.
- Структурируй текст подзаголовками (без раздела «Выводы»), выделяя примеры отдельными подпунктами.
- Если в тексте есть ФАКТЫ (имена, даты, места, события), используй их и укажи явно.
- Основанием служат ТОЛЬКО предоставленные тексты и ссылки.
"""
    return [
        {"role": "system", "content": sys},
        {"role": "user", "content": usr.strip()},
    ]


def ask_gpt(messages):
    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.2,
        max_tokens=5000,
    )
    return resp.choices[0].message.content.strip()


def collect_links(df_subset: pd.DataFrame, max_links: int = 3) -> list[str]:
    if link_col is None:
        return []
    links = (
        df_subset[link_col]
        .dropna()
        .astype(str)
        .drop_duplicates()
        .head(max_links)
        .tolist()
    )
    return links


def format_quotes_in_text(paragraph):
    """Форматирует цитаты: курсив, меньший размер, левое выравнивание и отступы"""
    text = paragraph.text
    quote_pattern = r'"([^"]+)"'
    matches = list(re.finditer(quote_pattern, text))
    if not matches:
        return

    paragraph.clear()
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    paragraph.paragraph_format.space_before = Pt(6)
    paragraph.paragraph_format.space_after = Pt(6)

    last_end = 0
    for match in matches:
        if match.start() > last_end:
            run = paragraph.add_run(text[last_end:match.start()])
            run.font.size = Pt(12)
            run.font.bold = False
        quote_run = paragraph.add_run(match.group(0))
        quote_run.font.italic = True
        quote_run.font.size = Pt(10)
        quote_run.font.bold = False
        last_end = match.end()

    if last_end < len(text):
        run = paragraph.add_run(text[last_end:])
        run.font.size = Pt(12)
        run.font.bold = False


def add_section(doc: Document, problem_key: str, problem_ru: str | None, analysis_text: str, links: list[str], is_first_section: bool = False):
    """Добавляет раздел в документ"""
    if not is_first_section:
        doc.add_page_break()

    heading = f"{problem_key}" + (f" — {problem_ru}" if problem_ru else "")
    h = doc.add_heading(heading, level=1)
    for r in h.runs:
        r.font.bold = True
        r.font.size = Pt(16)

    sections = analysis_text.split("\n\n")
    for section_text in sections:
        text_stripped = section_text.strip()
        if not text_stripped:
            continue
        if text_stripped.startswith("##"):
            subheading = text_stripped.replace("##", "").strip()
            if subheading.lower().startswith("вывод"):
                continue  # убираем раздел "Выводы"
            sh = doc.add_heading(subheading, level=2)
            for r in sh.runs:
                r.font.size = Pt(14)
                r.font.bold = True
        else:
            if text_stripped.lower().startswith("вывод"):
                continue  # на всякий случай, если пришло без ##
            p = doc.add_paragraph(text_stripped)
            p.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
            for r in p.runs:
                r.font.size = Pt(12)
                r.font.bold = False
            format_quotes_in_text(p)

    if links:
        doc.add_paragraph()
        links_heading = doc.add_paragraph("Источники и материалы:")
        for r in links_heading.runs:
            r.font.bold = True
            r.font.size = Pt(12)
        for url in links:
            link_para = doc.add_paragraph(url)
            for r in link_para.runs:
                r.font.size = Pt(10)
                r.font.bold = False
            link_para.paragraph_format.left_indent = Inches(0.5)


def create_table_of_contents(doc: Document, problems: list):
    """Создает оглавление"""
    h = doc.add_heading("Оглавление", level=1)
    for r in h.runs:
        r.font.bold = True
    for idx, (problem_key, problem_ru) in enumerate(problems, 1):
        toc_entry = f"{idx}. {problem_key}"
        if problem_ru:
            toc_entry += f" — {problem_ru}"
        p = doc.add_paragraph(toc_entry)
        for r in p.runs:
            r.font.size = Pt(12)
            r.font.bold = False
        p.paragraph_format.left_indent = Inches(0.25)


def create_research_summary(doc: Document, total_docs: int, date_range: str = None):
    """Создает описание исследования"""
    h = doc.add_heading("Описание исследования", level=1)
    for r in h.runs:
        r.font.bold = True
    summary_text = f"""
Настоящий отчет представляет результаты анализа сообщений жителей Танзании в социальных сетях, 
направленный на выявление ключевых проблем, волнующих население страны.

Параметры исследования:
• Дата создания отчета: {datetime.now().strftime('%d.%m.%Y')}
• Количество проанализированных документов: {total_docs}
• Количество тематических категорий: {len(df[problem_col].value_counts())}
• Основные источники: социальные сети, новостные порталы, блоги

Методология: качественный контент-анализ с использованием искусственного интеллекта для выявления 
ключевых тем и проблем в текстах сообщений.
    """
    p = doc.add_paragraph(summary_text.strip())
    p.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
    for r in p.runs:
        r.font.size = Pt(12)
        r.font.bold = False


def add_final_problem_tables(doc: Document):
    """Добавляет таблицу распределения проблем (final_problem_tables) после описания исследования"""
    # Заголовок таблицы
    h = doc.add_heading("Распределение проблем (final_problem_tables)", level=1)
    for r in h.runs:
        r.font.bold = True
    
    # Получаем распределение по русским названиям, если есть, иначе по исходным
    if problem_ru_col and problem_ru_col in df.columns:
        series = df[problem_ru_col].fillna(df[problem_col]).astype(str).value_counts()
    else:
        series = df[problem_col].astype(str).value_counts()
    
    # Создаем таблицу: заголовок + строки по категориям
    table = doc.add_table(rows=1, cols=2)
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = "Проблема"
    hdr_cells[1].text = "Количество"
    for p in hdr_cells:
        for rp in p.paragraphs:
            for run in rp.runs:
                run.font.bold = True
                run.font.size = Pt(12)
    
    for label, count in series.items():
        row_cells = table.add_row().cells
        row_cells[0].text = str(label)
        row_cells[1].text = str(int(count))
        for c in row_cells:
            for rp in c.paragraphs:
                for run in rp.runs:
                    run.font.bold = False
                    run.font.size = Pt(11)

# =============================
# Формирование отчёта
# =============================
problem_counts = df[problem_col].value_counts(dropna=True)
problems_ordered = list(problem_counts.index) #["Corruption"] 

# Создаем документ
document = Document()

# === ТИТУЛЬНАЯ СТРАНИЦА ===
section = document.sections[0]
section.page_height = Inches(11.69)  # A4
section.page_width = Inches(8.27)    # A4

# Заголовок
title = document.add_heading("Проблемы жителей Танзании: тематический анализ", 0)
title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

# Дата
date_para = document.add_paragraph(f"Дата формирования отчёта: {datetime.now().strftime('%d.%m.%Y')}")
date_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
for r in date_para.runs:
    r.font.size = Pt(14)
    r.font.bold = False

# Добавляем новую секцию для основного содержания
document.add_section(WD_SECTION.NEW_PAGE)

# === ОПИСАНИЕ ИССЛЕДОВАНИЯ ===
total_docs = len(df[~df[message_col].isna()])
create_research_summary(document, total_docs)

# === ТАБЛИЦА РАСПРЕДЕЛЕНИЯ ПРОБЛЕМ ===
add_final_problem_tables(document)

# === ОГЛАВЛЕНИЕ ===
document.add_page_break()
# Собираем информацию о проблемах для оглавления
toc_problems = []
for problem in problems_ordered:
    subset = df[df[problem_col] == problem].copy()
    if not subset.empty and problem_ru_col:
        problem_ru = subset[problem_ru_col].iloc[0] if pd.notna(subset[problem_ru_col].iloc[0]) else None
    else:
        problem_ru = None
    toc_problems.append((problem, problem_ru))

create_table_of_contents(document, toc_problems)

# Разрыв страницы после оглавления, чтобы следующий раздел начался с новой страницы
document.add_page_break()

# === ОСНОВНОЕ СОДЕРЖАНИЕ ===
for idx, problem in enumerate(problems_ordered):
    subset = df[df[problem_col] == problem].copy()
    subset = subset[~subset[message_col].isna()]
    if subset.empty:
        continue

    messages = (
        subset[message_col]
        .map(normalize_message)
        .dropna()
        .tolist()[:MAX_MESSAGES_PER_PROBLEM]
    )
    if not messages:
        continue

    problem_ru = (
        subset[problem_ru_col].iloc[0]
        if problem_ru_col and problem_ru_col in subset.columns and pd.notna(subset[problem_ru_col].iloc[0])
        else None
    )

    links = collect_links(subset, max_links=3)
    prompt_msgs = build_prompt(problem, problem_ru, messages, links)
    print(f"Анализ: {problem} ({len(messages)} сообщений)...")

    try:
        analysis = ask_gpt(prompt_msgs)
    except Exception as e:
        analysis = f"Не удалось получить анализ из модели. Ошибка: {e}"

    add_section(document, problem, problem_ru, analysis, links, is_first_section=(idx == 0))

# Колонтитулы и номера страниц (включая титульную)
add_header(document, "Проблемы жителей Танзании: тематический анализ")
add_page_numbers(document)

# Поля для всех секций кроме первой при необходимости
for section_idx, section in enumerate(document.sections):
    if section_idx > 0:
        section.top_margin = Inches(1)
        section.bottom_margin = Inches(1)
        section.left_margin = Inches(1)
        section.right_margin = Inches(1)

# Сохранение отчёта
document.save(OUTPUT_DOCX_PATH)
print(f"Отчёт сохранён: {OUTPUT_DOCX_PATH}")

Анализ: Poverty (15 сообщений)...
Анализ: Climate change (15 сообщений)...
Анализ: Corruption (15 сообщений)...
Анализ: Inflation (9 сообщений)...
Анализ: Access to water (6 сообщений)...
Анализ: Unemployment (6 сообщений)...
Анализ: Sanitation (5 сообщений)...
Анализ: Malnutrition (5 сообщений)...
Анализ: Inequality (4 сообщений)...
Анализ: Inadequate housing (4 сообщений)...
Анализ: Floods (4 сообщений)...
Анализ: Insecurity (4 сообщений)...
Анализ: Pollution (4 сообщений)...
Анализ: Mining exploitation (3 сообщений)...
Анализ: Child labor (2 сообщений)...
Анализ: Precarious employment (2 сообщений)...
Анализ: Access to education (2 сообщений)...
Отчёт сохранён: Tanzania_problems_report_20250929_1243.docx
